# 複数チャンネルの畳み込みとCNNの深層化

このノートブックでは、CNNが**複数の特徴マップ**をどのように処理するかを理解します。

## 目次
1. 1回目の畳み込み層の復習
2. プーリング層（1回目）
3. 2回目の畳み込み層（複数チャンネル入力）
4. なぜ複数チャンネルが必要？
5. パラメータ数の計算

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch
import matplotlib.patches as mpatches

## 1. 1回目の畳み込み層の復習

前回学んだ内容：

```
入力: 24×24×1（グレースケール画像）
  ↓ 5×5フィルタ × 2枚, stride=1
出力: 20×20×2（2枚の特徴マップ）
```

**ポイント**: 1枚の入力から**2枚**の特徴マップが生成された

In [ ]:
# 1回目の畳み込みの流れを図示
fig, ax = plt.subplots(figsize=(12, 4))

# 入力
ax.add_patch(FancyBboxPatch((0.5, 1), 2, 2, boxstyle="round,pad=0.05", 
                             facecolor='lightgray', edgecolor='black', linewidth=2))
ax.text(1.5, 2, '24×24×1\n(入力画像)', ha='center', va='center', fontsize=10)

# 矢印
ax.annotate('', xy=(3.5, 2), xytext=(2.7, 2), arrowprops=dict(arrowstyle='->', lw=2))
ax.text(3.1, 2.5, '5×5×2\nフィルタ', ha='center', fontsize=9)

# 出力（2枚の特徴マップ）
ax.add_patch(FancyBboxPatch((4, 1.2), 1.8, 1.6, boxstyle="round,pad=0.05", 
                             facecolor='steelblue', edgecolor='black', linewidth=2, alpha=0.7))
ax.add_patch(FancyBboxPatch((4.3, 0.9), 1.8, 1.6, boxstyle="round,pad=0.05", 
                             facecolor='steelblue', edgecolor='black', linewidth=2, alpha=0.9))
ax.text(5.2, 1.7, '20×20×2\n(特徴マップ\n2枚)', ha='center', va='center', fontsize=10, color='white')

ax.set_xlim(0, 8)
ax.set_ylim(0, 4)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('1回目の畳み込み層: 1チャンネル入力 → 2チャンネル出力', fontsize=12, fontweight='bold')
plt.show()

## 2. プーリング層（1回目）- 図3.12

畳み込み層の出力に対して、**Max Pooling**を適用します。

```
入力: 20×20×2
  ↓ 2×2 Max Pooling, stride=2
出力: 10×10×2
```

### 重要ポイント
- プーリングは**各チャンネル独立**に行われる
- チャンネル数は変わらない（2枚 → 2枚）
- サイズだけが半分になる（20×20 → 10×10）

### 記号
- $a^k_{i,j}$: プーリング後の特徴マップk枚目のi行j列目の値

In [ ]:
# 図3.12: プーリング層の処理を視覚化

fig, axes = plt.subplots(1, 3, figsize=(14, 5))

# 入力（20×20×2）
ax = axes[0]
# 2枚の特徴マップを重ねて表示
rect1 = FancyBboxPatch((0.1, 0.1), 0.8, 0.8, boxstyle="round,pad=0.02",
                        facecolor='steelblue', edgecolor='black', linewidth=2, alpha=0.6)
rect2 = FancyBboxPatch((0.2, 0), 0.8, 0.8, boxstyle="round,pad=0.02",
                        facecolor='steelblue', edgecolor='black', linewidth=2, alpha=0.9)
ax.add_patch(rect1)
ax.add_patch(rect2)
ax.text(0.5, -0.15, '$c^1$ (20×20)', ha='center', fontsize=10)
ax.text(0.6, -0.3, '$c^2$ (20×20)', ha='center', fontsize=10)
ax.set_xlim(-0.2, 1.2)
ax.set_ylim(-0.5, 1.1)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('入力\n20×20×2', fontsize=11)

# Max Pooling
ax = axes[1]
ax.text(0.5, 0.6, 'Max Pooling', ha='center', fontsize=14, fontweight='bold')
ax.text(0.5, 0.4, '2×2, stride=2', ha='center', fontsize=12)
ax.annotate('', xy=(0.9, 0.2), xytext=(0.1, 0.2),
            arrowprops=dict(arrowstyle='->', lw=3, color='red'))
ax.text(0.5, 0.0, '各チャンネル独立に処理\nチャンネル数は変わらない', ha='center', fontsize=10, style='italic')
ax.set_xlim(0, 1)
ax.set_ylim(-0.2, 0.8)
ax.axis('off')

# 出力（10×10×2）
ax = axes[2]
rect1 = FancyBboxPatch((0.15, 0.15), 0.6, 0.6, boxstyle="round,pad=0.02",
                        facecolor='lightblue', edgecolor='black', linewidth=2, alpha=0.6)
rect2 = FancyBboxPatch((0.25, 0.05), 0.6, 0.6, boxstyle="round,pad=0.02",
                        facecolor='lightblue', edgecolor='black', linewidth=2, alpha=0.9)
ax.add_patch(rect1)
ax.add_patch(rect2)
ax.text(0.45, -0.1, '$a^1$ (10×10)', ha='center', fontsize=10)
ax.text(0.55, -0.25, '$a^2$ (10×10)', ha='center', fontsize=10)
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-0.4, 1)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('出力\n10×10×2', fontsize=11)

plt.suptitle('図3.12: プーリング層（1回目）', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("プーリング層のポイント:")
print("  ・サイズ: 20×20 → 10×10（半分に圧縮）")
print("  ・チャンネル数: 2 → 2（変化なし）")
print("  ・各チャンネルは独立して処理される")

## 3. 2回目の畳み込み層（複数チャンネル入力）- 図3.13

ここが**重要**です！

```
入力: 10×10×2（2枚の特徴マップ）
  ↓ 3×3フィルタ × 4セット（各セット2枚）
出力: 8×8×4
```

### 複数チャンネル入力の処理方法

**入力が2チャンネルある場合、フィルタも2枚1セットになる！**

In [ ]:
# 複数チャンネルの畳み込みを詳しく図解

fig, ax = plt.subplots(figsize=(14, 8))

# === 入力（10×10×2）===
ax.add_patch(FancyBboxPatch((0.5, 5), 2, 2, boxstyle="round,pad=0.05",
                             facecolor='lightblue', edgecolor='black', linewidth=2, alpha=0.6))
ax.text(1.5, 6, '$a^1$\n(10×10)', ha='center', va='center', fontsize=10)
ax.add_patch(FancyBboxPatch((0.5, 2.5), 2, 2, boxstyle="round,pad=0.05",
                             facecolor='lightblue', edgecolor='black', linewidth=2, alpha=0.9))
ax.text(1.5, 3.5, '$a^2$\n(10×10)', ha='center', va='center', fontsize=10)
ax.text(1.5, 1.8, '入力\n10×10×2', ha='center', fontsize=11, fontweight='bold')

# === フィルタセット1（3×3×2）===
ax.add_patch(FancyBboxPatch((4, 5.2), 1, 1, boxstyle="round,pad=0.02",
                             facecolor='orange', edgecolor='black', linewidth=2, alpha=0.7))
ax.text(4.5, 5.7, '$w^{1,1}$\n(3×3)', ha='center', va='center', fontsize=8)
ax.add_patch(FancyBboxPatch((4, 3.8), 1, 1, boxstyle="round,pad=0.02",
                             facecolor='orange', edgecolor='black', linewidth=2, alpha=0.7))
ax.text(4.5, 4.3, '$w^{1,2}$\n(3×3)', ha='center', va='center', fontsize=8)

# 矢印（入力 → フィルタ）
ax.annotate('', xy=(3.9, 5.7), xytext=(2.6, 6), arrowprops=dict(arrowstyle='->', lw=1.5, color='gray'))
ax.annotate('', xy=(3.9, 4.3), xytext=(2.6, 3.5), arrowprops=dict(arrowstyle='->', lw=1.5, color='gray'))

# 積和演算の説明
ax.text(5.5, 5.7, '×', ha='center', va='center', fontsize=16)
ax.text(5.5, 4.3, '×', ha='center', va='center', fontsize=16)
ax.text(6.5, 5, '+', ha='center', va='center', fontsize=20, fontweight='bold', color='red')

# 計算結果
ax.add_patch(FancyBboxPatch((7.5, 4.5), 1.2, 1.2, boxstyle="round,pad=0.02",
                             facecolor='green', edgecolor='black', linewidth=2, alpha=0.7))
ax.text(8.1, 5.1, '$c^1_{1,1}$', ha='center', va='center', fontsize=12, color='white', fontweight='bold')

# 計算式
ax.text(10, 5.5, '$c^1_{1,1} = f($', ha='left', fontsize=11)
ax.text(10.2, 4.8, '$\sum w^{1,1} \cdot a^1$  ← 入力1との積和', ha='left', fontsize=10, color='blue')
ax.text(10.2, 4.3, '$+ \sum w^{1,2} \cdot a^2$  ← 入力2との積和', ha='left', fontsize=10, color='blue')
ax.text(10.2, 3.8, '$+ b_1$  ← バイアス', ha='left', fontsize=10, color='blue')
ax.text(10, 3.3, '$)$', ha='left', fontsize=11)

# 強調ボックス
ax.add_patch(FancyBboxPatch((3.5, 3.3), 2, 3.3, boxstyle="round,pad=0.1",
                             facecolor='none', edgecolor='red', linewidth=2, linestyle='--'))
ax.text(4.5, 2.8, 'フィルタセット1\n(3×3×2)', ha='center', fontsize=9, color='red')

ax.set_xlim(0, 15)
ax.set_ylim(1, 8)
ax.axis('off')
ax.set_title('図3.13: 2回目の畳み込み層 - 複数チャンネル入力の処理', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

### 計算の流れを言葉で説明

**出力 $c^1_{1,1}$ を計算するには：**

1. **入力チャンネル1** ($a^1$) の3×3領域と、**フィルタ1-1** ($w^{1,1}$) で積和演算
2. **入力チャンネル2** ($a^2$) の3×3領域と、**フィルタ1-2** ($w^{1,2}$) で積和演算  
3. 上記2つの結果を**合計**
4. **バイアス** $b_1$ を加算
5. **ReLU関数**を適用

$$c^1_{1,1} = f\left(\sum_{i,j} w^{1,1}_{i,j} \cdot a^1_{i,j} + \sum_{i,j} w^{1,2}_{i,j} \cdot a^2_{i,j} + b_1\right)$$

In [ ]:
# 具体的な数値例で計算してみる

np.random.seed(42)

# 入力（2チャンネル、各3×3領域）
a1 = np.array([[1, 2, 0],
               [0, 3, 1],
               [2, 1, 0]])

a2 = np.array([[0, 1, 2],
               [1, 0, 1],
               [0, 2, 1]])

# フィルタセット1（2枚のフィルタ）
w11 = np.array([[1, 0, -1],
                [1, 0, -1],
                [1, 0, -1]])

w12 = np.array([[1, 1, 1],
                [0, 0, 0],
                [-1, -1, -1]])

# バイアス
b1 = 0.5

print("=== 入力データ ===")
print(f"入力チャンネル1 (a¹):\n{a1}\n")
print(f"入力チャンネル2 (a²):\n{a2}\n")

print("=== フィルタセット1 ===")
print(f"フィルタ1-1 (w¹'¹):\n{w11}\n")
print(f"フィルタ1-2 (w¹'²):\n{w12}\n")

print("=== 計算過程 ===")
# 積和演算
sum1 = np.sum(a1 * w11)
sum2 = np.sum(a2 * w12)
print(f"Σ(a¹ × w¹'¹) = {sum1}")
print(f"Σ(a² × w¹'²) = {sum2}")

# 合計 + バイアス
total = sum1 + sum2 + b1
print(f"\n合計 + バイアス = {sum1} + {sum2} + {b1} = {total}")

# ReLU
output = max(0, total)
print(f"\nReLU({total}) = {output}")
print(f"\n∴ c¹₁,₁ = {output}")

## 4. なぜ複数チャンネルが必要？

### 各チャンネルは異なる特徴を捉える

| チャンネル | 捉える特徴の例 |
|-----------|---------------|
| チャンネル1 | 縦のエッジ |
| チャンネル2 | 横のエッジ |
| チャンネル3 | 斜めのエッジ |
| チャンネル4 | 角の検出 |

### 深い層では複雑な特徴を組み合わせる

```
浅い層: エッジ、線などの単純な特徴
  ↓
中間層: 形状、テクスチャなど
  ↓
深い層: 顔、物体などの複雑な概念
```

In [ ]:
# 複数フィルタの役割を視覚化

fig, axes = plt.subplots(2, 4, figsize=(14, 7))

# 典型的なフィルタパターン
filters = {
    '縦エッジ': np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]),
    '横エッジ': np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]]),
    '斜めエッジ': np.array([[0, -1, -1], [1, 0, -1], [1, 1, 0]]),
    'コーナー': np.array([[1, 0, -1], [0, 0, 0], [-1, 0, 1]])
}

for idx, (name, filt) in enumerate(filters.items()):
    ax = axes[0, idx]
    im = ax.imshow(filt, cmap='RdBu', vmin=-1, vmax=1)
    ax.set_title(f'フィルタ: {name}', fontsize=10)
    for i in range(3):
        for j in range(3):
            ax.text(j, i, str(filt[i, j]), ha='center', va='center', fontsize=12)
    ax.set_xticks([])
    ax.set_yticks([])

# 「8」の画像に各フィルタを適用した結果（イメージ）
sample_img = np.array([
    [0, 1, 1, 1, 0],
    [1, 0, 0, 0, 1],
    [0, 1, 1, 1, 0],
    [1, 0, 0, 0, 1],
    [0, 1, 1, 1, 0]
])

# 簡易的な畳み込み結果を表示
titles = ['縦エッジ検出', '横エッジ検出', '斜めエッジ検出', 'コーナー検出']
for idx, (name, filt) in enumerate(filters.items()):
    ax = axes[1, idx]
    # 簡易畳み込み（パディングなし）
    from scipy.signal import convolve2d
    result = convolve2d(sample_img, filt, mode='valid')
    ax.imshow(result, cmap='gray')
    ax.set_title(titles[idx], fontsize=10)
    ax.set_xticks([])
    ax.set_yticks([])

plt.suptitle('各フィルタは異なる特徴を捉える', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. パラメータ数の計算

### 2回目の畳み込み層

```
入力: 10×10×2
フィルタ: 3×3 × 4セット（各セット2枚）
出力: 8×8×4
```

**パラメータ数:**
- 重み: $3 \times 3 \times 2 \times 4 = 72$個
- バイアス: $4$個
- **合計: 76個**

### 一般化した公式

$$\text{パラメータ数} = (K_h \times K_w \times C_{in} \times C_{out}) + C_{out}$$

- $K_h, K_w$: フィルタの高さ・幅
- $C_{in}$: 入力チャンネル数
- $C_{out}$: 出力チャンネル数

In [ ]:
# パラメータ数の計算

def calc_conv_params(kernel_size, in_channels, out_channels):
    """畳み込み層のパラメータ数を計算"""
    weights = kernel_size[0] * kernel_size[1] * in_channels * out_channels
    biases = out_channels
    return weights, biases, weights + biases

print("=== CNNのパラメータ数 ===")
print()

# 1回目の畳み込み層
w, b, total = calc_conv_params((5, 5), 1, 2)
print(f"畳み込み層1 (5×5, 1ch→2ch):")
print(f"  重み: 5×5×1×2 = {w}")
print(f"  バイアス: {b}")
print(f"  合計: {total}")
print()

# 2回目の畳み込み層
w, b, total = calc_conv_params((3, 3), 2, 4)
print(f"畳み込み層2 (3×3, 2ch→4ch):")
print(f"  重み: 3×3×2×4 = {w}")
print(f"  バイアス: {b}")
print(f"  合計: {total}")

In [ ]:
# PyTorchで確認
import torch
import torch.nn as nn

# 2回目の畳み込み層を定義
conv2 = nn.Conv2d(in_channels=2, out_channels=4, kernel_size=3, stride=1)

print("PyTorchでの確認:")
print(f"  重みの形状: {conv2.weight.shape}  # (出力ch, 入力ch, H, W)")
print(f"  バイアスの形状: {conv2.bias.shape}")
print(f"  総パラメータ数: {sum(p.numel() for p in conv2.parameters())}")

# 動作確認
dummy_input = torch.randn(1, 2, 10, 10)  # バッチ, チャンネル, 高さ, 幅
output = conv2(dummy_input)
print(f"\n入力サイズ: {dummy_input.shape}")
print(f"出力サイズ: {output.shape}")

## 6. プーリング層（2回目）- 図3.14

2回目の畳み込み層の出力に対して、再びMax Poolingを適用します。

```
入力: 8×8×4
  ↓ 2×2 Max Pooling, stride=2
出力: 4×4×4
```

### 記号
- $z^k_{i,j}$: 2回目のプーリング後の特徴マップk枚目のi行j列目の値

### ポイント
- プーリングは**各チャンネル独立**に処理（1回目と同じ）
- チャンネル数は変わらない（4枚 → 4枚）
- サイズだけ半分になる（8×8 → 4×4）

In [ ]:
# 図3.14: プーリング層（2回目）の視覚化

fig, axes = plt.subplots(1, 3, figsize=(14, 5))

# 入力（8×8×4）
ax = axes[0]
colors = ['steelblue', 'green', 'orange', 'purple']
for i, color in enumerate(colors):
    offset = i * 0.08
    rect = FancyBboxPatch((0.1 + offset, 0.1 - offset), 0.7, 0.7, 
                          boxstyle="round,pad=0.02",
                          facecolor=color, edgecolor='black', linewidth=1.5, alpha=0.7)
    ax.add_patch(rect)
ax.text(0.5, -0.2, '$c^1, c^2, c^3, c^4$\n(8×8 × 4枚)', ha='center', fontsize=10)
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-0.4, 1)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('入力\n8×8×4', fontsize=11)

# Max Pooling
ax = axes[1]
ax.text(0.5, 0.6, 'Max Pooling', ha='center', fontsize=14, fontweight='bold')
ax.text(0.5, 0.4, '2×2, stride=2', ha='center', fontsize=12)
ax.annotate('', xy=(0.9, 0.2), xytext=(0.1, 0.2),
            arrowprops=dict(arrowstyle='->', lw=3, color='red'))
ax.text(0.5, 0.0, '各チャンネル独立に処理\n枚数は変わらない', ha='center', fontsize=10, style='italic')
ax.set_xlim(0, 1)
ax.set_ylim(-0.2, 0.8)
ax.axis('off')

# 出力（4×4×4）
ax = axes[2]
for i, color in enumerate(colors):
    offset = i * 0.06
    rect = FancyBboxPatch((0.2 + offset, 0.2 - offset), 0.5, 0.5, 
                          boxstyle="round,pad=0.02",
                          facecolor=color, edgecolor='black', linewidth=1.5, alpha=0.7)
    ax.add_patch(rect)
ax.text(0.5, -0.05, '$z^1, z^2, z^3, z^4$\n(4×4 × 4枚)', ha='center', fontsize=10)
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-0.3, 1)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('出力\n4×4×4', fontsize=11)

plt.suptitle('図3.14: プーリング層（2回目）', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("プーリング層（2回目）:")
print("  ・サイズ: 8×8 → 4×4（半分に圧縮）")
print("  ・チャンネル数: 4 → 4（変化なし）")
print("  ・出力の総要素数: 4×4×4 = 64個 → 全結合層へ")

## 7. CNN全体の流れまとめ

### 特徴マップの枚数について

**Q: 画像1枚につき特徴マップは何枚？**

**A: フィルタの枚数（出力チャンネル数）と同じ！**

| 層 | フィルタ数 | 特徴マップ数 |
|---|----------|------------|
| 畳み込み1 | 2枚 | 2枚 |
| 畳み込み2 | 4セット | 4枚 |

### 全体の流れ（図でおさらい）

In [ ]:
# CNN全体の流れを1枚の図にまとめる

fig, ax = plt.subplots(figsize=(16, 6))

# 各層の情報
layers = [
    {'name': '入力', 'size': '24×24×1', 'x': 0, 'color': 'lightgray', 'height': 2.4},
    {'name': '畳み込み1\n5×5×2', 'size': '20×20×2', 'x': 2.5, 'color': 'steelblue', 'height': 2.0},
    {'name': 'プーリング1\n2×2', 'size': '10×10×2', 'x': 5, 'color': 'lightblue', 'height': 1.0},
    {'name': '畳み込み2\n3×3×8', 'size': '8×8×4', 'x': 7.5, 'color': 'steelblue', 'height': 0.8},
    {'name': 'プーリング2\n2×2', 'size': '4×4×4', 'x': 10, 'color': 'lightblue', 'height': 0.4},
    {'name': '全結合', 'size': '64→10', 'x': 12.5, 'color': 'orange', 'height': 0.3},
    {'name': '出力', 'size': '10クラス', 'x': 15, 'color': 'green', 'height': 0.2},
]

for i, layer in enumerate(layers):
    # ボックスを描画
    h = layer['height']
    rect = FancyBboxPatch((layer['x'], 2 - h/2), 1.5, h, 
                          boxstyle="round,pad=0.05",
                          facecolor=layer['color'], edgecolor='black', linewidth=2, alpha=0.8)
    ax.add_patch(rect)
    
    # 層の名前
    ax.text(layer['x'] + 0.75, 2 + h/2 + 0.3, layer['name'], 
            ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    # サイズ
    ax.text(layer['x'] + 0.75, 2 - h/2 - 0.2, layer['size'], 
            ha='center', va='top', fontsize=9, color='darkblue')
    
    # 矢印
    if i < len(layers) - 1:
        ax.annotate('', xy=(layers[i+1]['x'] - 0.1, 2), 
                   xytext=(layer['x'] + 1.6, 2),
                   arrowprops=dict(arrowstyle='->', lw=2, color='gray'))

# 特徴マップ枚数を強調
ax.annotate('1枚', xy=(0.75, 0.3), fontsize=10, ha='center', color='red', fontweight='bold')
ax.annotate('2枚', xy=(3.25, 0.3), fontsize=10, ha='center', color='red', fontweight='bold')
ax.annotate('2枚', xy=(5.75, 0.3), fontsize=10, ha='center', color='red', fontweight='bold')
ax.annotate('4枚', xy=(8.25, 0.3), fontsize=10, ha='center', color='red', fontweight='bold')
ax.annotate('4枚', xy=(10.75, 0.3), fontsize=10, ha='center', color='red', fontweight='bold')

ax.set_xlim(-0.5, 17)
ax.set_ylim(-0.5, 4.5)
ax.axis('off')
ax.set_title('CNN全体の流れ: サイズと特徴マップ枚数の変化', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# テキストでもまとめ
print("=" * 60)
print("CNN全体の流れ")
print("=" * 60)
print()
print("入力画像      →  畳み込み1   →  プーリング1  →  畳み込み2   →  プーリング2  →  全結合  →  出力")
print("24×24×1         20×20×2        10×10×2        8×8×4         4×4×4         64→10     10クラス")
print("  (1枚)           (2枚)          (2枚)          (4枚)         (4枚)")
print()
print("【覚え方】")
print("  ・畳み込み: サイズ↓ 枚数はフィルタ数で決まる")
print("  ・プーリング: サイズ↓ 枚数は変わらない")

## まとめ

### プーリング層
- **各チャンネル独立**に処理
- チャンネル数は**変わらない**
- サイズだけ圧縮

### 複数チャンネル入力の畳み込み
- 入力がNチャンネルなら、フィルタも**N枚1セット**
- 各チャンネルとの積和演算を**合計**
- $c = f(\sum_k w^k \cdot a^k + b)$

### 特徴マップの枚数
- **フィルタの枚数 = 出力の特徴マップ枚数**
- 畳み込み1: フィルタ2枚 → 特徴マップ2枚
- 畳み込み2: フィルタ4セット → 特徴マップ4枚

### パラメータ数
$$\text{パラメータ数} = K_h \times K_w \times C_{in} \times C_{out} + C_{out}$$

## 次のステップ

次は**誤差逆伝播法**と**勾配降下法**による学習のメカニズムを学びます。